Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
# Imports libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the csv files from part 2 with Foursquare and Yelp EDA
df1 = pd.read_csv('fsq_bar_features.csv')
df2 = pd.read_csv('fsq_bar_features.csv')
df3 = pd.read_csv('yelp_bar_features.csv')

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.